출처: https://dacon.io/codeshare/3725?dtype=recent

In [178]:
import os
import glob
import json
from glob import glob
from tqdm import tqdm
import time
import datetime
import math


import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from collections import defaultdict, deque


import base64
import torch.distributed as dist
from torch.utils.data import Dataset

In [179]:
!ls

drive  sample_data


## 버전 확인

In [180]:
print('torch :', torch.__version__)
print('torchvision : ', torchvision.__version__)
print('cv2 :', cv2.__version__)

torch : 1.10.0+cu111
torchvision :  0.11.1+cu111
cv2 : 4.1.2


## 전체 데이터 json 만들기

In [181]:
# # 이건 저의 개인적인 코랩이기 때문에 다르신 분들은 다시 변경하시기 바랍니다.
# ## train_PATH와 test_PATH만 변경하면 됩니다.
# # json 파일로 만들어서 분석이 더 용이하다고 생각되어 만든 것이니 더 좋은 의견 있으면 말씀해주시기 바랍니다.

## Train
train_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/train/annotation'
file_list = os.listdir(train_PATH)

train_files = []
for file in tqdm(file_list):
    dir = train_PATH + '/' + file
    json_list = glob(dir + '/' +'*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

## Test
test_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/test/annotations'
file_list = os.listdir(test_PATH)

test_files = []
for file in tqdm(file_list):
    dir = test_PATH + '/' + file
    json_list = glob(dir + '/' + '*.json')
    test_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)

100%|██████████| 1000/1000 [00:00<00:00, 1019767.57it/s]

100%|██████████| 1000/1000 [00:00<00:00, 294792.24it/s]

100%|██████████| 1000/1000 [00:00<00:00, 912003.48it/s]

100%|██████████| 100/100 [00:00<00:00, 493447.53it/s]

100%|██████████| 100/100 [00:00<00:00, 535671.01it/s]

100%|██████████| 100/100 [00:00<00:00, 511500.49it/s]

100%|██████████| 4/4 [00:00<00:00, 143.83it/s]


## Sample data에서 json 만들기

In [182]:
## Train
train_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Sample_data/annotation'
file_list = os.listdir(train_PATH)

train_files = []
for file in tqdm(file_list):
    dir = train_PATH + '/' + file
    json_list = glob(dir + '/' +'*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

# 225로 한 이유? -> 252개 중에 0.8로 곱하거나 0.9로 곱하면 소숫점이 나오기 때문에 안됩니다 ㅜㅜ
# 그래서 임의로 한 것이기 때문에 더 완벽한 분석법이 있으면 알려주세요ㅜㅜ
train = train_json_list[:225]
test = train_json_list[225:]

  0%|          | 0/63 [00:00<?, ?it/s]Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7fb3fdf2cd40>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 109, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt
100%|██████████| 4/4 [01:23<00:00, 20.93s/it]


In [183]:
import os
import numpy as np
import torch
from PIL import Image

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms = None, mode = 'train', json_file = None):
        self.mode = mode
        self.root = root
        self.transforms = transforms
        self.json_file = json_file
        
    def __getitem__(self, idx):
        if self.mode == 'train':

            annot = json_file[idx]
            # Train의 경로로 이동한다.
            # 나중에 진짜 데이터를 다룰 때 사용한다.
            # PATH = root + '/' + 'train'

            with open(annot, 'r') as f:
                annot = json.loads(f.read())
                
            image_name = annot['images'][0]['file_name']
            dir = image_name.split('_')[0]

            image_PATH = PATH + '/' + dir + '/' + image_name
            
            image = Image.open(image_PATH).convert('RGB')

            boxes = []
            segmentations = []
            labels = []

            for i in range(len(annot['annotations'])):

                segmentation = annot['annotations'][i]['segmentation'][0]
                bbox = annot['annotations'][i]['bbox']
                label = torch.tensor(annot['annotations'][i]['category_id'], dtype = np.uint8)
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmin, ymin, xmax, ymax = xmin, ymin, xmin + width, ymax + height
                
                boxes.append([xmin, ymin, xmax, ymax])
                segmentations.append([segmentation])
                labels.append(label)

            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            target['segmentation'] = segmentations
            target['image_id'] = image_name

            if self.transforms is not None:
                img, target = self.transforms(img, target)

            return img, target

        if self.mode == 'test':
            # Test의 경로로 이동한다.
            # PATH = root + '/' + 'test'
            # 나중에 진짜 데이터를 다룰 때 사용한다.

            ##### 이 부분은 추후에 작성하기로 하자.
            with open(annot, 'r') as f:
                annot = json.loads(f.read())

            image_name = annot['images'][0]['file_name']
            dir = image_name[:2]

            image_PATH = PATH + '/' + dir + '/' + image_name
            image = Image.open(image_PATH).convert('RGB')
            target = {}

            target['image_id'] = image_name

            return image, target

    def __len__(self):
        return len(self.json_file)

In [184]:
root = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Sample_data'
train_dataset = CustomDataset(root = root,json_file = train, mode = 'train')
test_dataset = CustomDataset(root = root, json_file = test, mode = 'test')

In [185]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [186]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = 5, shuffle = True, num_workers = 16, collate_fn = collate_fn
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [187]:
def get_instance_segmentation_model(num_classes):

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained = True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

    return model

In [188]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# PP, PE, PET, PS, background
num_classes = 4 + 1

model = get_instance_segmentation_model(num_classes)

model.to(device)

params = [p for p in model.parameters() if p.requires_grad]

model.to(device)
optimizer = torch.optim.SGD(params, lr = 0.001, momentum = 0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.001)

In [189]:
class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)
    
class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        if torch.cuda.is_available():
            log_msg = self.delimiter.join([
                header,
                '[{0' + space_fmt + '}/{1}]',
                'eta: {eta}',
                '{meters}',
                'time: {time}',
                'data: {data}',
                'max mem: {memory:.0f}'
            ])
        else:
            log_msg = self.delimiter.join([
                header,
                '[{0' + space_fmt + '}/{1}]',
                'eta: {eta}',
                '{meters}',
                'time: {time}',
                'data: {data}'
            ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                if torch.cuda.is_available():
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time),
                        memory=torch.cuda.max_memory_allocated() / MB))
                else:
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time)))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))
        
def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):
    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)

def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict

def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()

def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True

In [190]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

In [191]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


IndexError: ignored